<a href="https://colab.research.google.com/github/bishair/Model/blob/main/model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import pandas as pd
import numpy as np
from google.colab import files
import os
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_squared_error
from keras.layers import Dropout
from keras.callbacks import EarlyStopping

In [2]:
#Function to determine if the user uploaded csv or xlsx file format
def load_data(file_name):
    # Determine the file extension
    file_extension = os.path.splitext(file_name)[1]
    # Load data based on file extension
    if file_extension == '.xlsx':
        return pd.read_excel(file_name)
    elif file_extension == '.csv':
        return pd.read_csv(file_name)
    else:
        raise ValueError("Unsupported file type. Please upload an Excel or CSV file.")

#Function that returns the index of the row where the header is located based on the provided keywords
def find_header_row(file_name, keywords):
    # Load only the first 10 rows of the file for efficiency
    data = load_data(file_name).head(10)
    # Iterate over the rows to find the header row
    for row_index, row in data.iterrows():
        # Iterate over each cell in the row. For each cell iterate over each keyword in keywords. Covert each cell to string and to lowercase for efficiency
        if any(keyword.lower() in str(cell).lower() for cell in row for keyword in keywords):
            # Return the index of the header row
            return row_index
    # If no header row is found, return None
    return None

#Function to determine which columns contain the data on GW level, date and river water level
def find_columns_by_keyword(dataframe, keywords):
    matched_columns = []
    for column in dataframe.columns:
        # Ensure the column name is a string before calling .lower()
        column_str = str(column)
        if any(keyword.lower() in column_str.lower() for keyword in keywords):
            matched_columns.append(column)
    return matched_columns


In [8]:
# Upload the original file
uploaded = files.upload()
#In case user upload more than one file, only take the name of the first file
input_file = next(iter(uploaded))

Saving data of 2014.xlsx to data of 2014 (1).xlsx


In [13]:
# Keywords to look for in the header row  =>  MODIFYY LATER DEPENDING UPON SAMPLE
header_keywords = ["date", "max", "min" ,"river", "water", "level", "groundwater", "G"]

In [14]:
# Find the header row
header_row = find_header_row(input_file, header_keywords)
print(header_row)

0


/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [59]:
# Check if the header row was found
if header_row is None:
    raise ValueError("Could not identify the header row.")

# Load the data with the correct header row
data = load_data(input_file)
print('Data', data)
print('****************************************************************************************')
print('Data columns before: ' , data.columns)
print('****************************************************************************************')
if header_row is not None:
    print('Reassigning header columns to row index 0',data.iloc[header_row])
    print('****************************************************************************************')
    data.columns = data.iloc[header_row]
    print('DATA columns after:', data.columns)
    print('****************************************************************************************')
    data = data.drop(data.index[0:header_row+1]).reset_index(drop=True)  #Remove all the rows above the header (header is removed as well). Reset the index to 0
    print('this is data',data)

Data               Unnamed: 0 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4  \
0                   date         G1         G4        G10        G11   
1    2014-06-02 00:00:00    110.738     110.72    110.757    110.739   
2    2014-06-03 00:00:00    110.658     110.64    110.672    110.659   
3    2014-06-04 00:00:00    110.563     110.54    110.577    110.564   
4    2014-06-05 00:00:00    110.498     110.46    110.512    110.484   
..                   ...        ...        ...        ...        ...   
129  2014-11-28 00:00:00        NaN        NaN    109.952    109.859   
130  2014-12-01 00:00:00        NaN        NaN    109.817    109.739   
131  2014-12-02 00:00:00        NaN        NaN    109.807    109.789   
132  2014-12-03 00:00:00        NaN        NaN    109.867    109.854   
133  2014-12-04 00:00:00        NaN        NaN    109.857    109.839   

    Unnamed: 5 Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9  ...  \
0          G12        G13        G15        G17        G19  ...

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [60]:
# Identify columns based on the keywords
date_columns = find_columns_by_keyword(data, ["date", "time"])
river_level_columns = find_columns_by_keyword(data, ["river", "water", "level", "max", "min"])
groundwater_level_columns = find_columns_by_keyword(data, ["groundwater", "G"])
print('Date: ', date_columns)
print('River Level: ', river_level_columns)
print('GW Level: ', groundwater_level_columns)

Date:  ['date']
River Level:  ['max', 'min']
GW Level:  ['G1', 'G4', 'G10', 'G11', 'G12', 'G13', 'G15', 'G17', 'G19', 'G21', 'G23']


In [61]:
# Check if we found exactly one column for each
if len(date_columns) != 1 or len(river_level_columns) < 2 or len(groundwater_level_columns) == 0:
    raise ValueError("Could not uniquely identify required columns. Please check the file format.")

# Extract the relevant columns
date_column = date_columns[0]
river_water_level_max = data[river_level_columns[0]].astype(float)  # Assuming first max
river_water_level_min = data[river_level_columns[1]].astype(float)  # Assuming second min
groundwater_level = data[groundwater_level_columns[0]].astype(float)  # Assuming first groundwater column  (MODDIFY LATER)
print('DATE COLUMN: ', date_column)
print('***********************************************************')
print('RIVER WATER LEVEL MAX: ', river_water_level_max)
print('***********************************************************')
print('RIVER WATER LEVEL MIN: ', river_water_level_min)
print('***********************************************************')
print('GW LEVEL: ', groundwater_level)

# Calculate the average river water level
river_water_level_avg = (river_water_level_max + river_water_level_min) / 2

# Handle missing values (if any) by forward filling
river_water_level_avg.fillna(method='ffill', inplace=True)
groundwater_level.fillna(method='ffill', inplace=True)

# Create a DataFrame with the preprocessed data
preprocessed_data = pd.DataFrame({
    'Date': pd.to_datetime(data[date_column]),
    'River_Water_Level_Avg': river_water_level_avg,
    'Groundwater_Level': groundwater_level
})

# Export the preprocessed data to a new Excel file
preprocessed_file_name = 'preprocessed_data_with_dates.xlsx'
preprocessed_data.to_excel(preprocessed_file_name, index=False)

DATE COLUMN:  date
***********************************************************
RIVER WATER LEVEL MAX:  0      110.659
1      110.490
2      110.391
3      110.292
4      110.234
        ...   
128    109.786
129    109.595
130    109.661
131    109.744
132    109.682
Name: max, Length: 133, dtype: float64
***********************************************************
RIVER WATER LEVEL MIN:  0      110.619
1      110.450
2      110.351
3      110.252
4      110.194
        ...   
128    109.746
129    109.555
130    109.621
131    109.704
132    109.642
Name: min, Length: 133, dtype: float64
***********************************************************
GW LEVEL:  0      110.738
1      110.658
2      110.563
3      110.498
4      110.428
        ...   
128        NaN
129        NaN
130        NaN
131        NaN
132        NaN
Name: G1, Length: 133, dtype: float64


In [62]:
# Download the preprocessed data file
files.download(preprocessed_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [63]:
#Load the pre-processed data
data = pd.read_excel('preprocessed_data_with_dates.xlsx')
print(data['River_Water_Level_Avg'].values)
print('**********************************************')
# Select the river water level as input and groundwater level as output. reshape(no. of rows based on arrays length, number of columns ) => 2 dimensional array 1 row 1 column for each element
input_data = data['River_Water_Level_Avg'].values.reshape(-1, 1)
output_data = data['Groundwater_Level'].values
print('Input Data',input_data)
print('**********************************************')
print('Output Data:', output_data)

[110.639 110.47  110.371 110.272 110.214 109.438 109.369 109.27  109.221
 109.082 109.112 109.098 109.1   109.092 108.995 109.118 109.069 109.18
 109.225 109.177 109.191 109.157 109.154 109.016 109.106 109.426 109.662
 109.77  109.372 109.298 109.253 109.16  109.147 109.089 109.14  109.55
 109.446 109.393 109.346 109.236 109.247 109.406 109.4   109.269 109.385
 109.334 109.479 109.595 109.544 109.316 109.233 109.209 109.136 109.236
 109.236 109.229 109.228 109.302 109.274 109.282 109.292 109.276 109.223
 109.174 109.235 109.227 109.321 109.358 109.395 109.364 109.528 109.704
 109.924 109.781 109.762 109.733 109.758 109.752 110.435 110.557 110.671
 110.436 110.386 110.455 110.473 110.418 110.383 109.961 110.127 110.139
 110.118 110.242 109.89  109.634 109.679 109.637 109.672 109.743 109.889
 109.908 109.955 110.127 110.143 110.312 110.655 110.897 110.757 110.716
 110.475 110.256 109.913 109.916 109.9   109.859 109.809 109.827 109.827
 109.904 109.909 109.88  109.795 109.79  110.087 110.

In [64]:
#Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_input_data = scaler.fit_transform(input_data)

In [65]:
#Create sequence for LSTM input
def create_sequences(input_data, output_data, time_steps):
    X, y = [], []
    for i in range(len(input_data) - time_steps):
        X.append(input_data[i:(i + time_steps)])
        y.append(output_data[i + time_steps])
    return np.array(X), np.array(y)

time_steps = 10  #Time steps for prediction
X, y = create_sequences(scaled_input_data, output_data, time_steps)

In [66]:
split = int(0.8 * len(X))  # 80% of data for training, 20% for testing
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [67]:
model = Sequential()
model.add(LSTM(units=2, return_sequences=True, input_shape=(time_steps, 1)))
model.add(Dropout(0.2))  # Dropout for regularization
model.add(LSTM(units=50))
model.add(Dropout(0.2))  # Dropout for regularization
model.add(Dense(1))

In [68]:
# Compile the RNN
model.compile(optimizer='adam', loss='mean_squared_error')

In [69]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

model.fit(X_train, y_train, epochs=100, batch_size=32,
          validation_data=(X_test, y_test),
          verbose=1, callbacks=[early_stopping])

Epoch 1/100
4/4 [==============================] - 5s 300ms/step - loss: 12038.2451 - val_loss: 12230.1797
Epoch 2/100
4/4 [==============================] - 0s 23ms/step - loss: 12028.5869 - val_loss: 12218.2637
Epoch 3/100
4/4 [==============================] - 0s 20ms/step - loss: 12016.6426 - val_loss: 12201.1787
Epoch 4/100
4/4 [==============================] - 0s 22ms/step - loss: 11998.4834 - val_loss: 12174.2275
Epoch 5/100
4/4 [==============================] - 0s 21ms/step - loss: 11969.4336 - val_loss: 12128.4990
Epoch 6/100
4/4 [==============================] - 0s 20ms/step - loss: 11920.4033 - val_loss: 12044.9062
Epoch 7/100
4/4 [==============================] - 0s 22ms/step - loss: 11823.7656 - val_loss: 11881.0879
Epoch 8/100
4/4 [==============================] - 0s 20ms/step - loss: 11639.9795 - val_loss: 11559.4072
Epoch 9/100
4/4 [==============================] - 0s 21ms/step - loss: 11275.8818 - val_loss: 11073.2070
Epoch 10/100
4/4 [===========================

In [70]:
# Make predictions
predicted_groundwater_level = model.predict(X_test)

# If you've scaled the data, you need to reverse the scaling to interpret the predictions
predicted_groundwater_level = scaler.inverse_transform(predicted_groundwater_level)

# Evaluate the model
mse = mean_squared_error(y_test, predicted_groundwater_level)
print(f"Mean Squared Error: {mse}")


1/1 [==============================] - 1s 795ms/step
Mean Squared Error: 3006.095569371479
